In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid
'''
def writeInfo(info, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_20PERCENT-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}{:.3f}_20PERCENT-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_20PERCENT-{}.txt".format(distanceMetric,clfName)
    file = open(path,"a") 
    file.write(info)
    file.close() 
'''
def writeResults(datasetID, bestScore, bestParams, clfName, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}{:.3f}_EVL-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}.txt".format(distanceMetric,clfName)
        
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main(distanceMetric, beta=None):
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    
    #arrClfName = ['SGD', 'NB', 'RF', 'LP', 'KNN']
    arrClfName = ['LP']
    if (distanceMetric == 'BBD') and (beta is None):
        raise Exception(("AMANDA-DCP with BBD must have a beta value. beta = {}").format(str(beta)))
    
    if beta is list:
        allBetas = True
    else:
        allBetas=False
    
    #writeInfo("**************** GRID SEARCH - 20% DOS DADOS ****************", distanceMetric, beta)
    
    for clfName in arrClfName:
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("**************** BEGIN of {}-{}{:.3f} results ****************".format(clfName, distanceMetric, beta))            
        else:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))

        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            splitPercentage = 0.2
            batches=16
            if i==len(datasets)-2: #NOAA
                batches=8
            elif i==len(datasets)-1: #Keystroke
                batches=4
                splitPercentage = 0.5
            
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)

            #Train-test split
            availableQty = int(splitPercentage*len(dataLabels))
            availableLabels = dataLabels[:availableQty] 
            availableData = dataValues[:availableQty]
            
            initialLabeledData = int(0.25*len(availableLabels))
            sizeOfBatch = int((len(availableLabels)-initialLabeledData)/batches)

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], 
                             "batches":[batches], "poolSize":[poolSize], "isBatchMode":[isBatchMode], 
                             "initialLabeledData":[initialLabeledData], "clfName":[clfName],
                            "distanceMetric":[distanceMetric], "beta":[beta]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(availableData, availableLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            writeResults(description, finalScore, best_grid, clfName, distanceMetric, beta)
        
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("******** END of {}-{}{:.3f} results ********".format(clfName, distanceMetric, beta))
        else:
            print("******** END of {}-{} results ********".format(clfName, distanceMetric))
    
if __name__ == "__main__":
    distanceMetric = 'BBD'
    betas = [-50, -1, -0.5, -0.1, 1.001, 1.1, 2, 3, 5, 10, 50]
    
    # Grid search for each beta value - BBD
    for beta in betas:
        main(distanceMetric, beta)
        print('\n\n\n')
        
    # grid search for Hellinger
    distanceMetric = 'Hellinger'
    main(distanceMetric, beta)
    print('\n\n\n')
    
    distanceMetric = 'Hellinger2'
    main(distanceMetric, beta)
    print('\n\n\n')


**************** BEGIN of LP-BBD-50.000 results ****************
One Class Diagonal Translation. 2 Dimensional data.: 16 batches of 150 instances
99.91624999999999 {'K': 2, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
99.833125 {'K': 3, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
99.91624999999999 {'K': 4, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
99.833125 {'K': 5, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
99.833125 {'K': 6, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, '

86.790625 {'K': 8, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
87.5825 {'K': 9, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
80.70875000000001 {'K': 10, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
61.54125 {'K': 11, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
86.833125 {'K': 12, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
87.5825
{'K': 9, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'is

53.945625 {'K': 3, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 225}
52.80625 {'K': 4, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 225}
58.7775 {'K': 5, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 225}
51.5275 {'K': 6, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 225}
84.55624999999999 {'K': 7, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 225}
70.19312500000001 {'K': 8, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData'

96.8775 {'K': 10, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
96.871875 {'K': 11, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
96.88499999999999 {'K': 12, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
96.88499999999999
{'K': 9, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
Two Bidimensional Multimodal Gaussian Classes.: 16 batches of 1875 instances
98.67375000000001 {'K': 2, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
98.68375 {'K': 3

97.429375 {'K': 4, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
97.536875 {'K': 5, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
97.56187499999999 {'K': 6, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
97.643125 {'K': 7, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
97.62062499999999 {'K': 8, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
97.54875 {'K': 9, 'batches': 16, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initi

...

85.333125 {'K': 5, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
54.917500000000004 {'K': 6, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
83.00125 {'K': 7, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
84.291875 {'K': 8, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
86.5825 {'K': 9, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 150}
85.91625 {'K': 10, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Helling

8.140625 {'K': 11, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 9150, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1715}
8.4725 {'K': 12, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 9150, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1715}
11.304375
{'K': 2, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 9150, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1715}
Five Classes Vertical Translation. Bidimensional.: 16 batches of 225 instances
46.916875000000005 {'K': 2, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 225}
49.77875 {'K': 3, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 225}
43

96.85125 {'K': 4, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
96.81875 {'K': 5, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
96.824375 {'K': 6, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
96.83125000000001 {'K': 7, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
96.85249999999999 {'K': 8, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 937}
96.86500000000001 {'K': 9, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'dist

99.815 {'K': 10, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
99.82125 {'K': 11, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
99.811875 {'K': 12, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
99.82125
{'K': 2, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
Two 5-dimensional Unimodal Gaussian Classes.: 16 batches of 1875 instances
97.314375 {'K': 2, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1875}
97.4168

63.22687500000001 {'K': 3, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 258}
64.1725 {'K': 4, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 258}
62.54875 {'K': 5, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 258}
64.583125 {'K': 6, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 258}
64.365 {'K': 7, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 258}
63.614999999999995 {'K': 8, 'batches': 16, 'beta': 50, 'clfName': 'LP', 'distanceMetric'

# 